# Import pandas and the dataset

In [4]:
import pandas as pd

In [25]:
toxic_x_test = pd.read_csv('test.csv',index_col='id')
toxic_y_test=pd.read_csv('test_labels.csv',index_col='id')
toxic_test=toxic_x_test.join(toxic_y_test, on='id')
toxic_test.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1
0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1
00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1
00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1


In [39]:
toxic_train = pd.read_csv('train.csv',index_col='id')
toxic_train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [40]:
nrow_train=toxic_train.shape[0]
nrow_test=toxic_test.shape[0]
sum=nrow_train+nrow_test
print("       : train : test")
print("rows   :",nrow_train,":",nrow_test)
print("perc   :",round(nrow_train*100/sum),"   :",round(nrow_test*100/sum))

       : train : test
rows   : 159571 : 153164
perc   : 51    : 49


It seems that the train data has more sample. To fix this issue, we shall combine those data and split it later when we are ready to apply ML model

In [43]:
toxic_df=pd.concat([toxic_test, toxic_train])
len(toxic_df)

312735

# Check for duplicate and null values

In [71]:
toxic_df[toxic_df.duplicated('comment_text')]

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,


In [73]:
toxic_df.isnull().sum()

comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

# Exploratory Data Analysis

Template for chart:

Add length of the comment to analyze

In [90]:
toxic_df['length'] = toxic_df['comment_text'].apply(len)
toxic_df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,length
id,,,,,,,,
00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1,367
0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1,50
00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1,54
00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1,205
00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1,41


## Data Validation

In [75]:
toxic_df.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,312735.000000,312735.000000,312735.000000,312735.000000,312735.000000,312735.000000
mean,-0.216803,-0.278907,-0.246362,-0.282978,-0.249035,-0.278411
std,0.553674,0.462240,0.513134,0.455311,0.509224,0.463074
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [70]:
toxic_df['threat'].drop_duplicates()

id
00001cee341fdb12   -1
0001ea8717f6de06    0
0081b14d79f54b31    1
Name: threat, dtype: int64

There seems to be three unique values in every single toxicity measure. Let's check a sample of each.

In [64]:
toxic_df[toxic_df['threat'] == -1]['comment_text'].iloc[3]

":If you have a look back at the source, the information I updated was the correct form. I can only guess the source hadn't updated. I shall update the information once again but thank you for your message."

In [65]:
toxic_df[toxic_df['threat'] == 0]['comment_text'].iloc[100]

"== No more April Fool'd edits by me in the article space. == \n\n I will not make any more April Fool's edits in the article space. \n\n Just in case you never saw it, here is my April Fool's edit from last year."

In [66]:
toxic_df[toxic_df['threat'] == 1]['comment_text'].iloc[100]

"Hi, Youri== \n\n *eyetwitch* I'm gonna kill you. \n\n As soon as you sign up for World of Warcraft. \n\n http://www.youtube.com/watch?v=6llCjpzHBCk \n\n YOU ARE SO DEAD-  \n\n =="

It seems like the ones with '1' is toxic while the rest is neutral. Replace -1 with 0 then convert the field to Boolean

## Data Visualization

## Begin cleaning the data by removing punctuation and stopwords

In [41]:
from nltk.corpus import stopwords

def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]